In [1]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

In [2]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased',return_token_type_ids = True)
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

In [26]:
context = "The US has passed the peak on new coronavirus cases, " \
          "President Donald Trump said and predicted that some states would reopen this month. " \
          "The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world."

questions = ["What was President Donald Trump's prediction?",
             "How many deaths have been reported from the virus?",
             "How many cases have been reported in the United States?"]

In [27]:
question_context_for_batch = []

for question in questions :
    question_context_for_batch.append((question, context))

In [28]:
encoding = tokenizer.batch_encode_plus(question_context_for_batch,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
start_scores, end_scores = model(input_ids, attention_mask=attention_mask)


In [29]:
for index,(start_score,end_score,input_id) in enumerate(zip(start_scores,end_scores,input_ids)):
    max_startscore = torch.argmax(start_score)
    max_endscore = torch.argmax(end_score)
    ans_tokens = input_ids[index][max_startscore: max_endscore + 1]
    answer_tokens = tokenizer.convert_ids_to_tokens(ans_tokens, skip_special_tokens=True)
    answer_tokens_to_string = tokenizer.convert_tokens_to_string(answer_tokens)
    print ("\nQuestion: ",questions[index])
    print ("Answer: ", answer_tokens_to_string)


Question:  What was President Donald Trump's prediction?
Answer:  some states would reopen this month

Question:  How many deaths have been reported from the virus?
Answer:  30 , 826

Question:  How many cases have been reported in the United States?
Answer:  over 637 , 000

Question:  Where is the United States
Answer:  the us has over 637 , 000 confirmed covid - 19 cases and over 30 , 826 deaths , the highest for any country in the world
